In [1]:
from boto3.session import Session
import boto3
from datetime import date
import numpy as np
from datetime import timedelta
import calendar
import awswrangler as wr
import pandas as pd
import io
import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import math
from statistics import NormalDist
from scipy.stats import norm
import sys

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
pd.set_option('display.float_format', '{:.9f}'.format)

In [4]:
#Question 1 data
s  = 30
vol = 0.3
q = 0.01
r = 0.025
t = 0.25
delta = 0.5
tol = 1e-6
op_type = 'call'

In [5]:
#Question 1
def q1_f(x):
    K = x
    d_1 = (math.log(s/K) + ((r - q + ((vol**2)/2))*t))/(vol*math.sqrt(t))
    N_d_1 = norm.cdf(d_1)
    return (math.exp(-q*t)*N_d_1) - (1/2)

def q1_fd(x):
    K = x
    d_1 = (math.log(s/K) + ((r - q + ((vol**2)/2))*t))/(vol*math.sqrt(t))
    n_d_1 = norm.pdf(d_1)
    t1 = 1/(K*vol*math.sqrt(t))
    return -1*math.exp(-q*t)*n_d_1*t1


In [6]:
#Question 1 Implementation
df_q1 = pd.DataFrame()
temp_row = {'Spot' : s, 'Volatility' :vol , 'dividendYield' : q , 'interestRate' : r , 'timeToExpiry' : t , 'Delta' : delta , 'x_(n)' : 0,'x_(n+1)' : 0}
x_old = sys.maxsize
x_new = s
i = 0
while abs(x_new - x_old) > tol or q1_f(x_new) > 1e-9:
    i += 1
    x_old = x_new
    x_new = x_old - (q1_f(x_old)/q1_fd(x_old))
    #print (i ,  x_new ,  q1_f(x_new))
    temp_row['x_(n)'] = x_old
    temp_row['x_(n+1)'] = x_new
    df_q1 = pd.concat([df_q1 , pd.DataFrame(temp_row , index = [0])], ignore_index = True)

In [7]:
df_q1

,Spot,Volatility,dividendYield,interestRate,timeToExpiry,Delta,x_(n),x_(n+1)
0,30,0.300000000,0.010000000,0.025000000,0.250000000,0.500000000,30.000000000,30.437314817
1,30,0.300000000,0.010000000,0.025000000,0.250000000,0.500000000,30.437314817,30.439064456
2,30,0.300000000,0.010000000,0.025000000,0.250000000,0.500000000,30.439064456,30.439064505


In [8]:
#Question 2 data
vol = 0.3
q = 0
r = 0.03
t = 0.5
#delta = 0.5
tol = 1e-6
op_type = 'call'

In [9]:
#Question 2 fx
def q2_f(x):
    d_1 = (math.log(x) + ((r - q + ((vol**2)/2))*t))/(vol*math.sqrt(t))
    d_2 = d_1 - (vol*math.sqrt(t))
    t1 = -1*(vol*x*math.exp(-(d_1**2)/2))/(2*math.sqrt(2*t*math.pi))
    t2 = r*math.exp(-r*t)*norm.cdf(-d_2)
    
    return t1 + t2 - 0

def q2_fd(x):
    d_1 = (math.log(x) + ((r - q + ((vol**2)/2))*t))/(vol*math.sqrt(t))
    d_2 = d_1 - (vol*math.sqrt(t))
    t1 = -1*(vol*math.exp(-(d_1**2)/2))/(2*math.sqrt(2*t*math.pi))
    t2 = (vol*x*math.exp(-(d_1**2)/2)*d_1)/ (2*math.sqrt(2*math.pi*t)*x*vol*math.sqrt(t))
    t3 = -1*(r*math.exp(-r*t)*norm.pdf(-d_2))/(x*vol*math.sqrt(t))
    
    return t1 + t2 + t3

In [10]:
#Question 2 Implementation
df_q2 = pd.DataFrame()
x_old = sys.maxsize
x_new = 0.8
temp_row = { 'Volatility' :vol , 'dividendYield' : q , 'interestRate' : r , 'timeToExpiry' : t , 'x_(n)' : 0,'x_(n+1)' : 0}
i = 0
while abs(x_new - x_old) > tol:
    i += 1
    x_old = x_new
    x_new = x_old - (q2_f(x_old)/q2_fd(x_old))
    temp_row['x_(n)'] = x_old
    temp_row['x_(n+1)'] = x_new
    df_q2 = pd.concat([df_q2 , pd.DataFrame(temp_row , index = [0])], ignore_index = True)
    #print (i ,  x_new ,  q2_f(x_new))

In [11]:
df_q2

,Volatility,dividendYield,interestRate,timeToExpiry,x_(n),x_(n+1)
0,0.300000000,0,0.030000000,0.500000000,0.800000000,0.738059292
1,0.300000000,0,0.030000000,0.500000000,0.738059292,0.734747020
2,0.300000000,0,0.030000000,0.500000000,0.734747020,0.734724245
3,0.300000000,0,0.030000000,0.500000000,0.734724245,0.734724244


In [12]:
#Question 3 data
#vol = 0.3
s= 30
K = 27
q = 0.01
r = 0.04
t = 7/12
#delta = 0.5
tol = 1e-6
op_type = 'call'
call_market = 4.5

In [13]:
#Question 3 functions
def q3_f(x):
    vol = x
    d_1 = (math.log(s/K) + ((r - q + ((vol**2)/2))*t))/(vol*math.sqrt(t))
    N_d_1 = norm.cdf(d_1)
    d_2 = d_1 - (vol*math.sqrt(t))
    N_d_2 = norm.cdf(d_2)
    call_p = s*(math.exp(-q*t))*N_d_1 - K*(math.exp(-r*t))*N_d_2
    return call_p - call_market

def q3_fp(x):
    vol = x
    d_1 = (math.log(s/K) + ((r - q + ((vol**2)/2))*t))/(vol*math.sqrt(t))
    #N_d_1 = norm.cdf(d_1)
    d_2 = d_1 - (vol*math.sqrt(t))
    #N_d_2 = norm.cdf(d_2)
    #call_p = s*(math.exp(-q*t))*N_d_1 - K*(math.exp(-r*t))*N_d_2
    N_m_d_1 = norm.cdf(-d_1)
    N_m_d_2 = norm.cdf(-d_2)
    put_p = K*(math.exp(-r*t))*N_m_d_2 - s*(math.exp(-q*t))*N_m_d_1
    return put_p - call_market

def q3_fd(x):
    vol = x
    t1 = s*math.exp(-q*t)*math.sqrt(t)
    d_1 = (math.log(s/K) + ((r - q + ((vol**2)/2))*t))/(vol*math.sqrt(t))
    n_d_1 = norm.pdf(d_1)
    return t1*n_d_1

In [19]:
#Question 3 Secant Implementation
df_q3_Sec = pd.DataFrame()
x_oldest = sys.maxsize
x_old = 0.5
x_new = 0.501
i = 0

while abs(x_new - x_old) > tol:
    temp_row = {'Spot' : s,  'dividendYield' : q , 'interestRate' : r , 'timeToExpiry' : t , 'x_n-1' : x_old ,'x_(n)' : 0,'x_(n+1)' : 0}
    i += 1
    print(i , x_old , x_new)
    x_newest = x_new - (q3_f(x_new)*(x_new- x_old))/(q3_f(x_new) - q3_f(x_old))
    x_old = x_new
    x_new = x_newest
    #print (i ,  x_new ,  q3_f(x_new))
    #temp_row['x_(n-1)'] = x_old
    temp_row['x_(n)'] = x_old
    temp_row['x_(n+1)'] = x_newest
    df_q3_Sec = pd.concat([df_q3_Sec , pd.DataFrame(temp_row , index = [0])], ignore_index = True)

1 0.5 0.501
2 0.501 0.2873289891429357
3 0.2873289891429357 0.28054889171594055
4 0.28054889171594055 0.2800918966696781
5 0.2800918966696781 0.28009006489673766


In [20]:
df_q3_Sec

,Spot,dividendYield,interestRate,timeToExpiry,x_n-1,x_(n),x_(n+1)
0,30,0.010000000,0.040000000,0.583333333,0.500000000,0.501000000,0.287328989
1,30,0.010000000,0.040000000,0.583333333,0.501000000,0.287328989,0.280548892
2,30,0.010000000,0.040000000,0.583333333,0.287328989,0.280548892,0.280091897
3,30,0.010000000,0.040000000,0.583333333,0.280548892,0.280091897,0.280090065
4,30,0.010000000,0.040000000,0.583333333,0.280091897,0.280090065,0.280090064


In [23]:
#Question 3 Newton Implementation
df_q3_new = pd.DataFrame()
x_old = sys.maxsize
x_new = 0.5
i = 0
temp_row = {'Spot' : s,  'dividendYield' : q , 'interestRate' : r , 'timeToExpiry' : t ,  'x_(n)' : 0,'x_(n+1)' : 0}
while abs(x_new - x_old) > tol:
    i += 1
    x_old = x_new
    x_new = x_old - (q3_f(x_old)/q3_fd(x_old))
    print (i ,  x_new ,  q3_f(x_new))
    temp_row['x_(n)'] = x_old
    temp_row['x_(n+1)'] = x_new
    #temp_row['x_(n+1)'] = x_newest
    df_q3_new = pd.concat([df_q3_new , pd.DataFrame(temp_row , index = [0])], ignore_index = True)

1 0.28731477840378905 0.05226681802651001
2 0.28011811498191447 0.0002021232792763783
3 0.2800900648668933 3.2218778756032407e-09
4 0.28009006441975565 3.552713678800501e-15


In [24]:
#df_q3_new = df_q3_new.round(9)
df_q3_new

,Spot,dividendYield,interestRate,timeToExpiry,x_(n),x_(n+1)
0,30,0.010000000,0.040000000,0.583333333,0.500000000,0.287314778
1,30,0.010000000,0.040000000,0.583333333,0.287314778,0.280118115
2,30,0.010000000,0.040000000,0.583333333,0.280118115,0.280090065
3,30,0.010000000,0.040000000,0.583333333,0.280090065,0.280090064


In [25]:
#Question 4 data
#vol = 0.3
s= 40
K = s
q = 0.01
r = 0.025
t = 5/12
#delta = 0.5
tol = 1e-6
op_type = 'call'
call_market = 2.75

In [26]:
#Question 4 Newton Implementation
df_q4_new = pd.DataFrame()
x_old = sys.maxsize
x_new = 0.5
i = 0
while abs(x_new - x_old) > tol:
    temp_row = {'Spot' : s,  'strike': K , 'dividendYield' : q , 'interestRate' : r , 'timeToExpiry' : t , 'price': call_market,  'x_(n)' : 0,'x_(n+1)' : 0}
    i += 1
    x_old = x_new
    x_new = x_old - (q3_f(x_old)/q3_fd(x_old))
    print (i ,  x_new ,  q3_f(x_new))
    temp_row['x_(n)'] = x_old
    temp_row['x_(n+1)'] = x_new
    df_q4_new = pd.concat([df_q4_new , pd.DataFrame(temp_row , index = [0])], ignore_index = True)

1 0.25565270113135363 -0.01273432224865445
2 0.25690315587560725 -1.6791767620816245e-07
3 0.2569031723648242 -3.552713678800501e-15


In [27]:
df_q4_new

,Spot,strike,dividendYield,interestRate,timeToExpiry,price,x_(n),x_(n+1)
0,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.500000000,0.255652701
1,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.255652701,0.256903156
2,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.256903156,0.256903172


In [28]:
#Question 4 Secant Implementation
df_q4_Sec = pd.DataFrame()
x_oldest = sys.maxsize
x_old = 0.5
x_new = 0.49
i = 0
while abs(x_new - x_old) > tol:
    temp_row = {'Spot' : s,  'strike': K , 'dividendYield' : q , 'interestRate' : r , 'timeToExpiry' : t , 'price': call_market, 'x_(n-1)':0, 'x_(n)' : 0,'x_(n+1)' : 0}
    i += 1
    print(i , x_old , x_new)
    x_newest = x_new - (q3_f(x_new)*(x_new- x_old))/(q3_f(x_new) - q3_f(x_old))
    x_old = x_new
    x_new = x_newest
    temp_row['x_(n-1)'] = x_old
    temp_row['x_(n)'] = x_old
    temp_row['x_(n+1)'] = x_newest
    df_q4_Sec = pd.concat([df_q4_Sec , pd.DataFrame(temp_row , index = [0])], ignore_index = True)
    print (i ,  x_new ,  q3_f(x_new))

1 0.5 0.49
1 0.2557149687150204 -0.012100204971961404
2 0.49 0.2557149687150204
2 0.2569075945777935 4.503353278551003e-05
3 0.2557149687150204 0.2569075945777935
3 0.2569031724204294 5.662528224092966e-10
4 0.2569075945777935 0.2569031724204294
4 0.2569031723648244 0.0


In [29]:
df_q4_Sec

,Spot,strike,dividendYield,interestRate,timeToExpiry,price,x_(n-1),x_(n),x_(n+1)
0,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.490000000,0.490000000,0.255714969
1,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.255714969,0.255714969,0.256907595
2,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.256907595,0.256907595,0.256903172
3,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.256903172,0.256903172,0.256903172


In [31]:
#Question 4 Bisection Method
df_q4_bi = pd.DataFrame()
a = 0.0001
b = 1
interval = b - a
while interval > tol:
    temp_row = {'Spot' : s,  'strike': K , 'dividendYield' : q , 'interestRate' : r , 'timeToExpiry' : t , 'price': call_market, 'a' : a , 'b' :b , 'mid': 0}
    mid = (a+b)/2
    temp_row['mid'] = mid
    _res = q3_f(mid)
    if _res == 0:
        break
    elif _res*q3_f(a) <0:
        b = mid
    else:
        a = mid
        
    interval = b - a
    print(a , b , interval , _res)
    temp_row['a'] = a
    temp_row['b'] = b
    
    df_q4_bi = pd.concat([df_q4_bi , pd.DataFrame(temp_row , index = [0])], ignore_index = True)

0.0001 0.50005 0.49995 2.4663675683897672
0.250075 0.50005 0.249975 -0.06953953226269505
0.250075 0.37506249999999997 0.12498749999999997 1.20135096138036
0.250075 0.31256874999999995 0.06249374999999996 0.5664886801689555
0.250075 0.28132187499999994 0.031246874999999952 0.24859898799169855
0.250075 0.2656984375 0.015623437500000004 0.08955788703364576
0.250075 0.25788671875 0.00781171875000003 0.010015824002383766
0.25398085937500003 0.25788671875 0.003905859374999987 -0.029760243550725818
0.25593378906250003 0.25788671875 0.0019529296874999935 -0.009871800650461893
0.25593378906250003 0.25691025390625 0.000976464843749969 7.211475819701718e-05
0.256422021484375 0.25691025390625 0.00048823242187501226 -0.004899817275365592
0.2566661376953125 0.25691025390625 0.00024411621093750613 -0.00241384482839635
0.25678819580078127 0.25691025390625 0.00012205810546872531 -0.001170863425990376
0.25684922485351563 0.25691025390625 6.1029052734362654e-05 -0.0005493739314275103
0.25687973937988284 

In [32]:
df_q4_bi

,Spot,strike,dividendYield,interestRate,timeToExpiry,price,a,b,mid
0,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.000100000,0.500050000,0.500050000
1,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.250075000,0.500050000,0.250075000
2,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.250075000,0.375062500,0.375062500
3,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.250075000,0.312568750,0.312568750
4,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.250075000,0.281321875,0.281321875
5,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.250075000,0.265698437,0.265698437
6,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.250075000,0.257886719,0.257886719
7,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.253980859,0.257886719,0.253980859
8,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.255933789,0.257886719,0.255933789
9,40,40,0.010000000,0.025000000,0.416666667,2.750000000,0.255933789,0.256910254,0.256910254


In [33]:
vt1 = ((r-q)/2)*t
vt2 = ((r+q)/2)*t
vt3 = math.sqrt(2*math.pi)
vt4 = s*math.sqrt(t)

sigma_imp_approx = (vt3/vt4)*((call_market - (vt1*s))/(1 - vt2))

q4_relative_error = (abs(sigma_imp_approx - 0.256903172))/0.256903172

In [34]:
print(sigma_imp_approx , q4_relative_error) 

0.25671024657084796 0.0007509655394681451


In [35]:
def calculate_ABC(y, R):
    A = (np.exp((1 - (2 / np.pi)) * y) - np.exp(-((1 - (2 / np.pi)) * y))) ** 2
    B = 4 * (np.exp((2 /np.pi) * y) + np.exp(-(2 / np.pi) * y)) - 2 * np.exp(-y) * (np.exp((1 - (2 / np.pi)) * y) + np.exp(-((1 - (2 / np.pi)) * y))) * (np.exp(2 * y) + 1 - R ** 2)
    C = np.exp(-2 * y) * ((R ** 2) - (np.exp(y) - 1) ** 2) * (np.exp(y) + 1) ** 2 - R
    return A, B, C

def approx_implied_vol_call(Cm, K, T, F, r):
    y = np.log(F / K)
    alphaC = Cm /np.exp(-r * T)
    R = 2 * alphaC - np.exp(y) + 1
    A, B, C = calculate_ABC(y, R)
    beta = 2 * C / (B + np.sqrt(B ** 2 + 4 * A * C))
    gamma = -np.pi / 2 * np.log(beta)
    if y >= 0:
        C0 = K * np.exp(-r * T) * (np.exp(y) * A * np.sqrt(2 * y) - 0.5)
        if Cm <= C0:
            sigma = 1 / np.sqrt(T) * (np.sqrt(gamma + y) - np.sqrt(gamma - y))
        else:
            sigma = 1 / np.sqrt(T) * (np.sqrt(gamma + y) + np.sqrt(gamma - y))
    else:
        C0 = K * np.exp(-r * T) * (np.exp(y) / 2 - A * (-np.sqrt(-2 * y)))
        print(y ,C0)
        if Cm <= C0:
            sigma = 1 / np.sqrt(T) * (-np.sqrt(gamma + y) + np.sqrt(gamma - y))
        else:
            sigma = 1 / np.sqrt(T) * (np.sqrt(gamma + y) + np.sqrt(gamma - y))
    return sigma

def approx_implied_vol_put(Pm, K, T, F, r):
    y = np.log(F / K)
    alphaP = Pm * np.exp(-r * T)
    R = 2 * alphaP + np.exp(y) - 1
    A, B, C = calculate_ABC(y, R)
    beta = 2 * C / (B + np.sqrt(B ** 2 + 4 * A * C))
    gamma = -np.pi / 2 * np.log(beta)
    if y >= 0:
        P0 = K * np.exp(-r * T) * (0.5 - np.exp(y) * A * (-np.sqrt(2 * y)))
        if Pm <= P0:
            sigma = 1 / np.sqrt(T) * (np.sqrt(gamma + y) - np.sqrt(gamma - y))
        else:
            sigma = 1 / np.sqrt(T) * (np.sqrt(gamma + y) + np.sqrt(gamma - y))
    else:
        P0 = K * np.exp(-r * T) * (A * np.sqrt(-2 * y) - np.exp(y) / 2)
        if Pm <= P0:
            sigma = 1 / np.sqrt(T) * (-np.sqrt(gamma + y) + np.sqrt(gamma - y))
        else:
            sigma = 1 / np.sqrt(T) * (np.sqrt(gamma + y) + np.sqrt(gamma - y))
    return sigma

In [99]:
import numpy as np

def calculate_ABC(y, R):
    A = ((np.exp(((1 - (2 / np.pi)) * y)) - np.exp(-((1 - (2 / np.pi)) * y))) ** 2)
    B = (4 * (np.exp(((2/np.pi) * y)) + np.exp(-(((2 / np.pi) * y)))) - (2 * np.exp(-y) * ((np.exp((1 - (2 / np.pi)) * y) + np.exp(-((1 - (2 / np.pi)) * y))) * (np.exp((2 * y)) + 1) - (R ** 2))))
    C = (np.exp((-2) * y) * ((R ** 2 - ((np.exp(y) - 1) ** 2)) * ((np.exp(y) + 1) ** 2) - R**2))
    print(A,B,C)
    return A, B, C

def calculate_ABC(y, R):
    # Compute terms for A
    term1_A = math.exp((1 - (2/math.pi)) * y)
    term2_A = math.exp(-(1 - (2/math.pi)) * y)
    A = (term1_A - term2_A) ** 2

    # Compute terms for B
    term1_B = 4 * (math.exp((2/math.pi) * y) + math.exp(-(2/math.pi) * y))
    term2_B = 2 * math.exp(-y) * (math.exp((1 - (2/math.pi)) * y) + math.exp(-(1 - (2/math.pi)) * y))
    term3_B = (math.exp(2 * y) + 1 - (R ** 2))
    B = term1_B - (term2_B * term3_B)

    # Compute terms for C
    term1_C = math.exp(-2 * y)
    term2_C = (R ** 2) - ((math.exp(y) - 1) ** 2)
    term3_C = ((math.exp(y) + 1) ** 2) - (R ** 2)
    C = term1_C * term2_C * term3_C

    return A, B, C

def A_f(x):
    common_term = 0.5 * np.sqrt(1 - np.exp(-2 * x**2) / np.pi)
    if x >= 0:
        return 0.5 + common_term
    else:
        return 0.5 - common_term

def approx_implied_vol_call(Cm, K, T, F, r):
    y = np.log(F / K)
    alphaC = Cm/(K*np.exp(-r * T))
    R = ((2 * alphaC) - np.exp(y) + 1)
    A, B, C = calculate_ABC(y, R)
    beta = ((2 * C) / (B + np.sqrt((B ** 2) + (4 * A * C))))
    gamma = (-(np.pi / 2) * np.log(beta))
    if y >= 0:
        C0 = K * np.exp(-r * T) * ((np.exp(y) * (A_f(np.sqrt((2 * y))))) - 0.5)
        if Cm <= C0:
            sigma = (1 / np.sqrt(T)) * ((np.sqrt(gamma + y)) - np.sqrt((gamma - y)))
        else:
            sigma = (1 / np.sqrt(T)) * ((np.sqrt(gamma + y)) + np.sqrt((gamma - y)))
    else:
        C0 = K * np.exp(-r * T) * ((np.exp(y) / 2) - A_f(-np.sqrt((-2) * y)))
        if Cm <= C0:
            sigma = (1 / np.sqrt(T)) * ((-np.sqrt(gamma + y)) + np.sqrt((gamma - y)))
        else:
            sigma = (1 / np.sqrt(T)) * ((np.sqrt(gamma + y)) + np.sqrt((gamma - y)))
    return sigma

def approx_implied_vol_put(Pm, K, T, F, r):
    print(Pm, K, T, F, r)
    y = np.log(F / K)
    alphaP = Pm/(K*np.exp(-r * T))
    R = ((2 * alphaP) + np.exp(y) - 1)
    A, B, C = calculate_ABC(y, R)
    print(A,B,C)
    beta = ((2 * C) / (B + np.sqrt((B ** 2) + (4 * A * C))))
    gamma = (-(np.pi / 2) * np.log(beta))
    print(beta , gamma)
    if y >= 0:
        P0 = K * np.exp(-r * T) * (0.5 - (np.exp(y) * A_f(-np.sqrt((2 * y)))))
        print(y,P0)
        if Pm <= P0:
            sigma = (1 / np.sqrt(T)) * ((np.sqrt(gamma + y)) - np.sqrt((gamma - y)))
        else:
            sigma = (1 / np.sqrt(T)) * ((np.sqrt(gamma + y)) + np.sqrt((gamma - y)))
    else:
        P0 = K * np.exp(-r * T) * ((A_f(np.sqrt((-2) * y))) - ((np.exp(y) / 2)))
        if Pm <= P0:
            sigma = (1 / np.sqrt(T)) * ((-np.sqrt(gamma + y)) + np.sqrt((gamma - y)))
        else:
            sigma = (1 / np.sqrt(T)) * ((np.sqrt(gamma + y)) + np.sqrt((gamma - y)))
    return sigma


In [37]:
print(call_market, K, t, forward, r)
approx_implied_vol_put(call_market, K, t, forward, r)

NameError: name 'forward' is not defined

In [38]:
import math

def compute_abc(y, R):
    A = (math.exp((1 - 2/math.pi) * y) - math.exp(-(1 - 2/math.pi) * y)) ** 2
    B = 4 * (math.exp(2/math.pi * y) + math.exp(-2/math.pi * y)) - 2 * math.exp(-y) * (math.exp((1 - 2/math.pi) * y) + math.exp(-(1 - 2/math.pi) * y)) * (math.exp(2 * y) + 1 - R ** 2)
    C = math.exp(-2 * y) * (R ** 2 - (math.exp(y) - 1) ** 2) * ((math.exp(y) + 1) ** 2 - R ** 2)
    return A, B, C

def implied_volatility(market_price, K, T, F, r, call=True):
    y = math.log(F / K)
    alpha = market_price / (K * math.exp(-r * T))
    R = 2 * alpha - math.exp(y) + 1 if call else 2 * alpha + math.exp(y) - 1
    A, B, C = compute_abc(y, R)
    beta = 2 * C / (B + math.sqrt(B ** 2 + 4 * A * C))
    gamma = -math.pi / 2 * math.log(beta)
    if y >= 0:
        C0 = K * math.exp(-r * T) * (math.exp(y * A * math.sqrt(2 * y)) - 0.5) if call else K * math.exp(-r * T) * (0.5 - math.exp(y * A * (-math.sqrt(2 * y))))
        if market_price <= C0:
            sigma = math.sqrt(1 / T) * (math.sqrt(gamma) + y - math.sqrt(gamma - y))
        else:
            sigma = math.sqrt(1 / T) * (math.sqrt(gamma) + y + math.sqrt(gamma - y))
    else:
        C0 = K * math.exp(-r * T) * (math.exp(y / 2) - A * math.sqrt(-2 * y)) if call else K * math.exp(-r * T) * (A * math.sqrt(-2 * y) - math.exp(y / 2))
        if market_price <= C0:
            sigma = math.sqrt(1 / T) * (-math.sqrt(gamma) + y + math.sqrt(gamma - y))
        else:
            sigma = math.sqrt(1 / T) * (math.sqrt(gamma) + y + math.sqrt(gamma - y))
    return sigma

In [39]:
implied_volatility(call_market, K, t, forward, r, call=True)

NameError: name 'forward' is not defined

In [100]:
#Question 5 data
import pandas as pd

data_q5 = {
    'Contract': ['C1175', 'C1200', 'C1225', 'C1250', 'C1275', 'C1300', 'C1325', 'C1350', 'C1375', 'C1400', 'C1425', 'C1450', 'C1500', 'C1550', 'C1575', 'C1600', 'P1175', 'P1200', 'P1225', 'P1250', 'P1275', 'P1300', 'P1325', 'P1350', 'P1375', 'P1400', 'P1425', 'P1450', 'P1500', 'P1550', 'P1575', 'P1600'],
    'Type': ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
    'Strike': ['1175', '1200', '1225', '1250', '1275', '1300', '1325', '1350', '1375', '1400', '1425', '1450', '1500', '1550', '1575', '1600', '1175', '1200', '1225', '1250', '1275', '1300', '1325', '1350', '1375', '1400', '1425', '1450', '1500', '1550', '1575', '1600'],
    'Price': [225.40, 205.55, 186.20, 167.50, 149.15, 131.70, 115.25, 99.55, 84.90, 71.10, 58.70, 47.25, 29.25, 15.80, 11.10, 7.90, 46.60, 51.55, 57.15, 63.30, 70.15, 77.70, 86.20, 95.30, 105.30, 116.55, 129.00, 143.20, 173.95, 210.80, 230.90, 252.40]
}

df_q5 = pd.DataFrame(data_q5)
df_q5['Strike'] = df_q5['Strike'].astype(int) 
df_q5['Spot'] = 1370
df_q5['timeToExpiry'] = (199/252)
df_q5['interestRate'] = 0.004567
df_q5['dividendYield'] = 0.019054
df_q5['forward'] = df_q5['Spot']*np.exp((df_q5['interestRate'] - df_q5['dividendYield'])*df_q5['timeToExpiry'] )
df_q5['impliedVolatility'] = None
df_q5['approxImpliedVolatility'] = None

df_q5

,Contract,Type,Strike,Price,Spot,timeToExpiry,interestRate,dividendYield,forward,impliedVolatility,approxImpliedVolatility
0,C1175,C,1175,225.400000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,None,None
1,C1200,C,1200,205.550000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,None,None
2,C1225,C,1225,186.200000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,None,None
3,C1250,C,1250,167.500000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,None,None
4,C1275,C,1275,149.150000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,None,None
5,C1300,C,1300,131.700000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,None,None
6,C1325,C,1325,115.250000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,None,None
7,C1350,C,1350,99.550000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,None,None
8,C1375,C,1375,84.900000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,None,None
9,C1400,C,1400,71.100000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,None,None


In [101]:
j = 0
for index, row in df_q5.iterrows():
    j += 1
    print('Row Number: ' , j)
    s, K, t, r , q = row['Spot'], row['Strike'], row['timeToExpiry'] , row['interestRate'] , row['dividendYield']
    call_market = row['Price']
    op_type = row['Type']
    forward = s*math.exp((r-q)*t)
    tol = 1e-6
    if op_type == 'C':
        f5 = q3_f
        approx_f = approx_implied_vol_call
    else:
        f5 = q3_fp
        approx_f = approx_implied_vol_put
    
    r_q_n = ((r-q)/2)*t
    r_q_d = ((r+q)/2)*t
    ini_guess = (math.sqrt(2*math.pi)/(s*math.sqrt(t)))*((call_market-(r_q_n*s))/(1-r_q_d))
    x_old = sys.maxsize
    x_new = 0.25 #ini_guess
    i = 0
    
    while abs(x_new - x_old) > tol:
        i += 1
        x_old = x_new
        x_new = x_old - (f5(x_old)/q3_fd(x_old))
        #print (i ,  x_new ,  f5(x_new))
    
    df_q5.at[index , 'impliedVolatility'] = x_new
    approx_f(call_market, K, t, forward, r)
    df_q5.at[index , 'approxImpliedVolatility'] = approx_f(call_market, K, t, forward, r)

df_q5['relativeError'] = abs(df_q5['approxImpliedVolatility'] - df_q5['impliedVolatility'])/df_q5['impliedVolatility']

Row Number:  1
Row Number:  2
Row Number:  3
Row Number:  4
Row Number:  5
Row Number:  6
Row Number:  7
Row Number:  8
Row Number:  9
Row Number:  10
Row Number:  11
Row Number:  12
Row Number:  13
Row Number:  14
Row Number:  15
Row Number:  16
Row Number:  17
46.6 1175 0.7896825396825397 1354.416330168137 0.004567
0.010675094889069441 0.1285839275974574 0.10564500094714194
0.7721104526837985 0.40625138739399264
0.14210246129153023 521.5338688083085
46.6 1175 0.7896825396825397 1354.416330168137 0.004567
0.010675094889069441 0.1285839275974574 0.10564500094714194
0.7721104526837985 0.40625138739399264
0.14210246129153023 521.5338688083085
Row Number:  18
51.55 1200 0.7896825396825397 1354.416330168137 0.004567
0.007744358363881599 0.12112247870706305 0.1043063980947156
0.8183458611424332 0.3148978830198277
0.12104905209369798 528.0568553768773
51.55 1200 0.7896825396825397 1354.416330168137 0.004567
0.007744358363881599 0.12112247870706305 0.1043063980947156
0.8183458611424332 0.3148

In [102]:
df_q5

,Contract,Type,Strike,Price,Spot,timeToExpiry,interestRate,dividendYield,forward,impliedVolatility,approxImpliedVolatility,relativeError
0,C1175,C,1175,225.400000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,0.257273413,0.255027171,0.008730953
1,C1200,C,1200,205.550000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,0.249599424,0.247910962,0.006764687
2,C1225,C,1225,186.200000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,0.241899316,0.240693237,0.004985871
3,C1250,C,1250,167.500000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,0.234400382,0.233599776,0.003415548
4,C1275,C,1275,149.150000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,0.226261260,0.225784393,0.002107594
5,C1300,C,1300,131.700000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,0.218604610,0.218369863,0.001073839
6,C1325,C,1325,115.250000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,0.211441536,0.211362499,0.000373800
7,C1350,C,1350,99.550000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,0.204054328,0.040539650,0.801329133
8,C1375,C,1375,84.900000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,0.196910679,0.194042600,0.014565381
9,C1400,C,1400,71.100000000,1370,0.789682540,0.004567000,0.019054000,1354.416330168,0.189417531,0.189242524,0.000923919


In [ ]:
from py_vollib.black_scholes.implied_volatility import implied_volatility as iv

In [ ]:
iv(4.5, 30, 27, 7/12, 0.04, 'c')

In [43]:
#question 6
def q6_f(X):
    x = X[0][0]
    y = X[1][0]
    print(x,y)
    return (3*(x**2)*y) + (x**2) - (6*x) - (3*y) - 2
def q6_gf(X):
    x = X[0][0]
    y = X[1][0]
    
    grad_1 = (6*x*y) + (2*x) - 6
    grad_2 = (3*(x**2)) - 3
    
    grad = np.array([grad_1 , grad_2]).reshape(-1,1)
    return grad

def q6_Hf(X):
    x = X[0][0]
    y = X[1][0]
    #print(x,y)
    h11 = (6*y) + 2 
    h12 = 6*x
    h21 = h12
    h22 = 0
    
    hessian = np.array([[h11,h12] , [h21, h22]])
    return hessian

In [52]:
#Question 6 Newton Implementation
df_q6 = pd.DataFrame()
x_old = np.array([sys.maxsize,sys.maxsize]).reshape(-1,1)
x_new = np.array([[-0.5],[-0.5]])#.reshape(-1,1)
i = 0
tol = 1e-6
tol = np.array([tol,tol]).reshape(-1,1)
#temp_row = {'x_(n)' : x_old[0][0] , 'x_(n+1)' : x_new[0][0] , 'y_(n)' : x_old[1][0] , 'y_(n+1)' : x_new[1][0]}
#df_q6 = pd.concat([df_q6 , pd.DataFrame(temp_row , index = [0])], ignore_index = True)
while (abs(x_new - x_old) > tol).any():
    
    i += 1
    x_old = x_new
    x_new = x_old - np.matmul((np.linalg.inv(q6_Hf(x_old))),(q6_gf(x_old)))
    print (i)
    print("X_NEW:  " , x_new)
    print("F(x):  " , q6_f(x_new))
    print("GRAD F(x)" , q6_gf(x_new))
    temp_row = {'x_(n)' : x_old[0][0] ,'y_(n)' : x_old[1][0],  'x_(n+1)' : x_new[0][0]   , 'y_(n+1)' : x_new[1][0]}
    df_q6 = pd.concat([df_q6 , pd.DataFrame(temp_row , index = [0])], ignore_index = True)

1
X_NEW:   [[-1.25      ]
 [-2.08333333]]
-1.25 -2.083333333333333
F(x):   3.546875000000001
GRAD F(x) [[7.125 ]
 [1.6875]]
2
X_NEW:   [[-1.025     ]
 [-1.44833333]]
-1.025 -1.4483333333333333
F(x):   4.980659375
GRAD F(x) [[0.85725 ]
 [0.151875]]
3
X_NEW:   [[-1.00030488]
 [-1.33580656]]
-1.0003048780487804 -1.3358065635534404
F(x):   4.99999519625684
GRAD F(x) [[0.01667317]
 [0.00182955]]
4
X_NEW:   [[-1.00000005]
 [-1.33333404]]
-1.0000000464611474 -1.3333340405752576
F(x):   4.999999999999796
GRAD F(x) [[4.52221863e-06]
 [2.78766891e-07]]
5
X_NEW:   [[-1.        ]
 [-1.33333333]]
-1.000000000000001 -1.3333333333333652
F(x):   5.0
GRAD F(x) [[1.98951966e-13]
 [6.66133815e-15]]


In [53]:
df_q6

,x_(n),y_(n),x_(n+1),y_(n+1)
0,-0.500000000,-0.500000000,-1.250000000,-2.083333333
1,-1.250000000,-2.083333333,-1.025000000,-1.448333333
2,-1.025000000,-1.448333333,-1.000304878,-1.335806564
3,-1.000304878,-1.335806564,-1.000000046,-1.333334041
4,-1.000000046,-1.333334041,-1.000000000,-1.333333333


In [55]:
#Question 6 Newton Implementation
df_q6_2 = pd.DataFrame()
x_old = np.array([sys.maxsize,sys.maxsize]).reshape(-1,1)
x_new = np.array([[0.5],[0.5]])#.reshape(-1,1)
i = 0
tol = 1e-6
tol = np.array([tol,tol]).reshape(-1,1)
#temp_row = {'x' : x_new[0][0] , 'y' : x_new[1][0]}
#df_q6_2 = pd.concat([df_q6_2 , pd.DataFrame(temp_row , index = [0])], ignore_index = True)
while (abs(x_new - x_old) > tol).any():
    
    i += 1
    x_old = x_new
    x_new = x_old - np.matmul((np.linalg.inv(q6_Hf(x_old))),(q6_gf(x_old)))
    print (i)
    print("X_NEW:  " , x_new)
    print("F(x):  " , q6_f(x_new))
    print("GRAD F(x)" , q6_gf(x_new))
    #temp_row = {'x' : x_new[0][0] , 'y' : x_new[1][0]}
    temp_row = {'x_(n)' : x_old[0][0] ,'y_(n)' : x_old[1][0],  'x_(n+1)' : x_new[0][0]   , 'y_(n+1)' : x_new[1][0]}
    df_q6_2 = pd.concat([df_q6_2 , pd.DataFrame(temp_row , index = [0])], ignore_index = True)

1
X_NEW:   [[1.25      ]
 [0.41666667]]
1.25 0.41666666666666685
F(x):   -7.234375
GRAD F(x) [[-0.375 ]
 [ 1.6875]]
2
X_NEW:   [[1.025     ]
 [0.60166667]]
1.025 0.6016666666666667
F(x):   -7.007996875
GRAD F(x) [[-0.24975 ]
 [ 0.151875]]
3
X_NEW:   [[1.00030488]
 [0.66480319]]
1.0003048780487804 0.6648031925441207
F(x):   -7.000003130461886
GRAD F(x) [[-0.00935499]
 [ 0.00182955]]
4
X_NEW:   [[1.00000005]
 [0.66666605]]
1.0000000464611474 0.6666660523470366
F(x):   -7.000000000000164
GRAD F(x) [[-3.40715107e-06]
 [ 2.78766891e-07]]
5
X_NEW:   [[1.        ]
 [0.66666667]]
1.000000000000001 0.6666666666666371
F(x):   -7.0
GRAD F(x) [[-1.70530257e-13]
 [ 6.66133815e-15]]


In [56]:
df_q6_2

,x_(n),y_(n),x_(n+1),y_(n+1)
0,0.500000000,0.500000000,1.250000000,0.416666667
1,1.250000000,0.416666667,1.025000000,0.601666667
2,1.025000000,0.601666667,1.000304878,0.664803193
3,1.000304878,0.664803193,1.000000046,0.666666052
4,1.000000046,0.666666052,1.000000000,0.666666667


In [105]:
q6_array = -np.array([[6,6] , [6,0]])
np.linalg.cholesky(q6_array)

LinAlgError: Matrix is not positive definite

In [106]:
q6_array

array([[-6, -6],
       [-6,  0]])

In [58]:
#question 6
def q7_f(X):
    x1 = X[0][0]
    x2 = X[1][0]
    x3 = X[2][0]
    x3 = X[2][0]
    lam  = X[3][0]
    return x1**2 + 3*x2**2 + x3**2 - 2*x1*x2 - x2*x3 + x1*x3 + lam * (x1**2 + x2**2 + x3**2 - 6)

def q7_gf(X):
    x1 = X[0][0]
    x2 = X[1][0]
    x3 = X[2][0]
    lam  = X[3][0]
    
    return np.array([
        2*x1 - 2*x2 + x3 + 2*lam*x1,
        6*x2 - 2*x1 - x3 + 2*lam*x2,
        2*x3 - x2 + x1 + 2*lam*x3,
        x1**2 + x2**2 + x3**2 - 6
    ]).reshape(-1,1)
    
    
    

def q7_Hf(X):
    x1 = X[0][0]
    x2 = X[1][0]
    x3 = X[2][0]
    lam  = X[3][0]
    
    return np.array([
        [2 + 2*lam, -2, 1, 2*x1],
        [-2, 6 + 2*lam, -1, 2*x2],
        [1, -1, 2 + 2*lam, 2*x3],
        [2*x1, 2*x2, 2*x3, 0]
    ])
    

In [86]:
#Question 7 Newton Implementation
df_q7 = pd.DataFrame()
x_old = np.array([sys.maxsize,sys.maxsize , sys.maxsize , sys.maxsize]).reshape(-1,1)
x_new = np.array([2, 0.5 , -1,0.5]).reshape(-1,1)
i = 0
tol = 1e-6
tol = np.array([tol,tol,tol,tol]).reshape(-1,1)
#temp_row = {'x1' : x_new[0][0] , 'x2' : x_new[1][0] , 'x3' : x_new[2][0] , 'lambda' : x_new[3][0]}
#df_q7 = pd.concat([df_q7 , pd.DataFrame(temp_row , index = [0])], ignore_index = True)
while (abs(x_new - x_old) > tol).any():
    i += 1
    x_old = x_new
    x_new = x_old - np.matmul((np.linalg.inv(q7_Hf(x_old))),(q7_gf(x_old)))
    #print (i , '\n')
    #print("X_NEW: \n  " , x_new)
    #print("F(x):  " , q6_f(x_new))
    #print("GRAD F(x)" , q6_gf(x_new))
    temp_row = {'x1_(n)' : x_old[0][0] , 'x2_(n)' : x_old[1][0] , 'x3_(n)' : x_old[2][0] , 'lambda_(n)' : x_old[3][0] ,
                'x1_(n+1)' : x_new[0][0] , 'x2_(n+1)' : x_new[1][0] , 'x3_(n+1)' : x_new[2][0] , 'lambda_(n+1)' : x_new[3][0]}
    df_q7 = pd.concat([df_q7 , pd.DataFrame(temp_row , index = [0])], ignore_index = True)

print("Hessian")
q7_Hf(x_new)
chol_decomp = np.linalg.cholesky(q7_Hf(x_new))

Hessian


LinAlgError: Matrix is not positive definite

In [87]:
#Try different initial guess
print("")
display(df_q7)
print("\n\nHESSIAN\n\n")
print(q7_Hf(x_new))
print(np.linalg.cholesky(q7_Hf(x_new)))


,x1_(n),x2_(n),x3_(n),lambda_(n),x1_(n+1),x2_(n+1),x3_(n+1),lambda_(n+1)
0,2.000000000,0.500000000,-1.000000000,0.500000000,2.087402344,0.571289062,-1.164550781,-0.488769531
1,2.087402344,0.571289062,-1.164550781,-0.488769531,2.016437765,0.529152808,-1.295334670,-0.418847353
2,2.016437765,0.529152808,-1.295334670,-0.418847353,2.020243520,0.535184732,-1.277714661,-0.418861143
3,2.020243520,0.535184732,-1.277714661,-0.418861143,2.020182871,0.535168794,-1.277675833,-0.418861170
4,2.020182871,0.535168794,-1.277675833,-0.418861170,2.020182870,0.535168794,-1.277675832,-0.418861170




HESSIAN


[[ 1.16227766 -2.          1.          4.04036574]
 [-2.          5.16227766 -1.          1.07033759]
 [ 1.         -1.          1.16227766 -2.55535166]
 [ 4.04036574  1.07033759 -2.55535166  0.        ]]


LinAlgError: Matrix is not positive definite

In [62]:
print(np.linalg.cholesky(-q7_Hf(x_new)))

LinAlgError: Matrix is not positive definite